In [ ]:
# import settings and functions
%run ./../../dataset_creation_imports.ipynb

## What mesh?

Copy your choice to the next cell

for SquareTop:
```
analytical_solution_tag = "-ana_square_top"
generate_config = generateConfig_squareTop
generate_mesh = generateMesh_squareTop
```

for SquareSinCos:
```
analytical_solution_tag = "-ana_square_sincos"
generate_config = generateConfig_squareSinCos
generate_mesh = generateMesh_squareSinCos
```

In [ ]:
# Change according to instruction above
analytical_solution_tag = "-ana_square_sincos"
generate_config = generateConfig_squareSinCos
generate_mesh = generateMesh_squareSinCos

# analytical_solution_tag = "-ana_L_shape"
# generate_config = generateConfig_Lshape
# generate_mesh = generateMesh_Lshape

# analytical_solution_tag = "-ana_mexi_hat"
# generate_config = generateConfig_squareMexiHat
# generate_mesh = generateMesh_squareMexiHat

# analytical_solution_tag = "-ana_square_top"
# generate_config = generateConfig_squareTop
# generate_mesh = generateMesh_squareTop

In [ ]:
how_many = 41
#
params.conductivity = 1.0 # linear conductivity
params.element_size = 0.05 # element size in the regular mesh
params.order = 2 # approximation order for displacements
# Pre-processing parameters
params.mesh_file = "square_top"
params.length_x = 1
params.length_y = 1

params.nproc = 1 # number of processors

In [ ]:
# start display for showing results
display = Display(backend="xvfb", visible=False, size=(1024, 768))
display.start()

In [ ]:
# Testing mesh generation

params.show_mesh = True
generate_config(params)
generate_mesh(params)

In [ ]:
dataset_name = "grid_data.csv"
make_grid_linear_dataset(8, how_many, dataset_name, plot_dataset=True)

In [ ]:
# Testing running analysis
!rm out_*

use_line = "-use_line"
use_line = ""

how_many = 41
make_grid_linear_dataset(8, how_many, dataset_name, plot_dataset=False)

params.part_file = params.mesh_file + "_" + str(params.nproc) + "p.h5m"
!{mofem_part} -my_file {params.mesh_file + ".h5m"} -my_nparts {params.nproc} -output_file {params.part_file} -dim 2 -adj_dim 1
!{data_driven_diffusion_snes} -file_name {params.part_file} -my_order {params.order} {analytical_solution_tag} -csv_tree_file {dataset_name} -write_long_error_file {use_line} -print_integ
# data_driven_diffusion
# hdiv_data_driven_diffusion

out_to_vtk = !ls -c1 out_result_*h5m

!convert.py {out_to_vtk[0]}

In [ ]:

params.show_file = "out_iteration_"
params.show_file = "out_result_"
params.show_field = "T"
# params.warp_factor = 0.4  # warp factor
params.show_edges = True
# params.p_save = "run_test_p.pdf"
show_results(params)

In [ ]:
params.show_field = "Q"
params.field_part = 1
show_results(params)

In [ ]:
point_file = "out_integ_pts_6"

!mbconvert {point_file + ".h5m"} {point_file + ".vtk"}

params.show_file = "out_integ_pts_"
params.show_field = "GRAD(P)_STAR"
params.field_part = 1
# params.warp_factor = 0.4  # warp factor
params.show_edges = True
# params.p_save = "run_test_p.pdf"
show_resulting_points(params)

In [ ]:
params.show_field = "T"
# params.warp_factor = 0.4  # warp factor
# params.p_save = "run_test_p.pdf"
show_resulting_points(params)

In [ ]:
def save_point_fields(vtk_file, point_index, csv_file):
    # Read the .vtk file
    mesh = pv.read(vtk_file)

    # Get the point
    point = mesh.points[point_index]

    # Get the fields (scalars) of the point
    fields = mesh.point_data

    # Prepare data for saving
    data_to_save = {field: fields[field][point_index] for field in fields}

    # Save the fields to a .csv file
    with open(csv_file, 'w') as f:
        writer = csv.DictWriter(f, fieldnames=data_to_save.keys())
        writer.writeheader()

In [ ]:
# import csv
# point_file = "out_integ_pts_6"

# !mbconvert {point_file + ".h5m"} {point_file + ".vtk"}
# save_point_fields(point_file+".vtk", 0, "point_fields.csv")

In [ ]:
import csv
import glob
import pandas as pd
import matplotlib.pyplot as plt
import pyvista as pv

!convert.py out_integ_pts_*

def get_point_fields(directory, point_index):
    # Get the list of .vtk files
    vtk_files = sorted(glob.glob(f"{directory}/out_integ_pts_*.vtk"))

    # Initialize a DataFrame to store the fields
    df = None

    for vtk_file in vtk_files:
        # Read the .vtk file
        mesh = pv.read(vtk_file)

        # Get the fields (scalars) of the point
        fields = mesh.point_data

        # Prepare data for saving
        data_to_save = {}
        for field in fields:
            if np.isscalar(fields[field][point_index]):
                # Scalar field
                data_to_save[field] = fields[field][point_index]
            elif len(fields[field][point_index].shape) == 1:
                # Vector field
                for i in range(fields[field][point_index].shape[0]):
                    data_to_save[f"{field}_{i}"] = fields[field][point_index][i]

        # Add the file number to the data
        file_number = int(vtk_file.split('_')[-1].split('.')[0])
        data_to_save['file_number'] = file_number

        # Append the data to the DataFrame
        if df is None:
            df = pd.DataFrame([data_to_save])
        else:
            df = pd.concat([df, pd.DataFrame([data_to_save])], ignore_index=True)

    return df

# plot_point_fields(".", 20)
df_gauss_vlaues = get_point_fields(".", 20)


In [ ]:
print(df_gauss_vlaues)

In [ ]:
dataset_name = "grid_data.csv"
grid_data = pd.read_csv(dataset_name)

In [ ]:
# Interleave the points from 'fields' and 'stars'
x_values = np.ravel(np.column_stack((df_gauss_vlaues['GRAD(P)_0'], df_gauss_vlaues['GRAD(P)_STAR_0'])))
y_values = np.ravel(np.column_stack((df_gauss_vlaues['Q_0'], df_gauss_vlaues['Q_STAR_0'])))


# Calculate the differences between consecutive points
dx = np.diff(x_values)
dy = np.diff(y_values)

# Calculate the length of each line segment
lengths = np.hypot(dx, dy)

plt.figure()
plt.scatter(grid_data['gradx'], grid_data['fluxx'], label='Material dataset', zorder=1, s=2)
plt.scatter(df_gauss_vlaues['GRAD(P)_0'], df_gauss_vlaues['Q_0'], label='Field values', marker='x', zorder=1)
plt.scatter(df_gauss_vlaues['GRAD(P)_STAR_0'], df_gauss_vlaues['Q_STAR_0'], label='Closest data point', marker='*', zorder=1, color='b')

# Normalize the lengths to get widths
widths = lengths / np.max(lengths) * 0.005  # adjust the factor as needed

# Plot each arrow individually with its own width
for i in range(len(x_values[:-1])):
    plt.quiver(x_values[i], y_values[i], dx[i], dy[i], scale_units='xy', angles='xy', scale=1, color='black', width=widths[i], zorder=2)

# plt.xlabel('Greadient')
# plt.ylabel('Flux')
plt.xlabel(r'Temperature gradient')
plt.ylabel(r'Heat flux')
# plt.xlim(-0.5,8)
# plt.ylim(-8,0.5)
ax = plt.gca()  # Define ax variable
ax.grid(True, ls=':')  # Call ax.grid() method
plt.legend()
plt.tight_layout()
plt.savefig('integration_journey.pdf')
plt.show()


In [ ]:
data = pd.read_csv('errors_long_file.csv', header=0,  index_col=False)

In [ ]:
fig, ax = plt.subplots()
ax.plot(data['rmsErr_long_file'], label=r'Temperature error')
ax.plot(data[' rmsGradErr_long_file'], label=r'Gradient error')
ax.plot(data[' rmsFluxErr_long_file'], label=r'Flux error')
ax.plot(data[' rmsPointDistErr_long_file'], label=r'Point distance', ls='--', color='black', marker='o')
ax.set_yscale('log')
ax.grid(True, ls=':')
ax.legend()

## Plot with line and with dataset

In [ ]:
main_error_file = "errors_long_file.csv"

line_errors = "line_errors.csv"
data_errors = "data_errors.csv"

In [ ]:
use_line = "-use_line"
use_line = ""

how_many = 101
make_grid_linear_dataset(8, how_many, dataset_name, plot_dataset=False)

params.part_file = params.mesh_file + "_" + str(params.nproc) + "p.h5m"
!{mofem_part} -my_file {params.mesh_file + ".h5m"} -my_nparts {params.nproc} -output_file {params.part_file} -dim 2 -adj_dim 1
# !{classic_diffusion} -file_name {params.part_file} -my_order {params.order} {analytical_solution_tag}
!{data_driven_diffusion_snes} -file_name {params.part_file} -my_order {params.order} {analytical_solution_tag} -csv_tree_file {dataset_name} -write_long_error_file {use_line}

!mv {main_error_file} {data_errors}

In [ ]:
# use_line = "-use_line"

# params.part_file = params.mesh_file + "_" + str(params.nproc) + "p.h5m"
# !{mofem_part} -my_file {params.mesh_file + ".h5m"} -my_nparts {params.nproc} -output_file {params.part_file} -dim 2 -adj_dim 1
# # !{classic_diffusion} -file_name {params.part_file} -my_order {params.order} {analytical_solution_tag}
# !{data_driven_diffusion_snes} -file_name {params.part_file} -my_order {params.order} {analytical_solution_tag} -csv_tree_file {dataset_name} -write_long_error_file {use_line}

# !mv {main_error_file} {line_errors}

In [ ]:
line_pd = pd.read_csv(line_errors, header=0,  index_col=False)
data_pd = pd.read_csv(data_errors, header=0,  index_col=False)

fig, ax = plt.subplots()
ax.plot(data_pd['rmsErr_long_file'], label=r'Temperature error')
ax.plot(data_pd[' rmsGradErr_long_file'], label=r'Gradient error')
ax.plot(data_pd[' rmsFluxErr_long_file'], label=r'Flux error')
ax.plot(data_pd[' rmsPointDistErr_long_file'], label=r'Point distance', ls='--', color='black', marker='o')
ax.set_ylabel(r'Global RMS error')
ax.set_xlabel(r'Iteration')
ax.set_yscale('log')
ax.grid(True, ls=':')
y_range = ax.get_ylim()
ax.legend()
plt.tight_layout()
fig.savefig("DD_small_data_errors_with_iterations.pdf")

fig, ax = plt.subplots()
ax.plot(line_pd['rmsErr_long_file'], label=r'Temperature error')
ax.plot(line_pd[' rmsGradErr_long_file'], label=r'Gradient error')
ax.plot(line_pd[' rmsFluxErr_long_file'], label=r'Flux error')
ax.plot(line_pd[' rmsPointDistErr_long_file'], label=r'Point distance', ls='--', color='black', marker='o')
ax.set_ylabel(r'Global RMS error')
ax.set_xlabel(r'Iteration')
ax.set_yscale('log')
ax.grid(True, ls=':')
ax.legend()
ax.set_ylim(y_range)
plt.tight_layout()
fig.savefig("DD_line_errors_with_iterations.pdf")
